## Reading and Merging CSV Files

In [15]:
import pandas as pd
import datetime
import glob
import os

# Assuming your CSV files are in the same directory and have a common pattern
path = your files path
csv_files = glob.glob(path + "/*.csv")

room_dfs = {}

for dirname, _,filenames in os.walk(path):

    if len(filenames) > 1:

        # Get the number of each room
        room_id = str(dirname).split('\\')[-1]

        csv_files = glob.glob(dirname + "/*.csv")

        # Define a dictionary to store DataFrames
        dfs = {}   

        for file in csv_files:
            key = (str(file).split('\\')[-1]).split('.')[0]
    
            # Read CSV and create DataFrame
            df = pd.read_csv(file)

            # Eacg key is sensor name
            column_names = ['time', key]

            df.columns = column_names

            # Converting the Unix Epoch Time to time stamps
            df['time'] = pd.to_datetime(df['time'],unit='s')
            
            # Setting the second values to 0
            df['time'] = df['time'].dt.floor('T')

            # Group by time column and calculate mean value of sensor datas
            df = df.groupby('time').mean().reset_index()

            # Changing column positions
            df = df[['time', key]]

            # Store DataFrame in the dictionary with sensor names
            dfs["df_" + key] = df

        # Merge all DataFrames based on the 'time' column
        room_df = pd.concat([df.set_index('time') for df in dfs.values()], axis=1, keys=dfs.keys())
        room_id = str(dirname).split('\\')[-1]
        room_df['Room_ID'] = room_id

        room_df.reset_index(inplace = True)
        room_df.columns = ["time", "co2", "humidity", "light", "pir", "temp", "room"]

        # reposition columns
        room_df = room_df[["time", "room", "co2", "light", "temp", "humidity",  "pir"]]

        # add room dfs to dictionary
        room_dfs["df_" + room_id] = room_df

In [16]:
# Merge them all
concatenated_df = pd.concat(room_dfs.values(), ignore_index=True)
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602650 entries, 0 to 602649
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   time      602650 non-null  datetime64[ns]
 1   room      602650 non-null  object        
 2   co2       548776 non-null  float64       
 3   light     548593 non-null  float64       
 4   temp      548594 non-null  float64       
 5   humidity  548593 non-null  float64       
 6   pir       588598 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 32.2+ MB


In [18]:
concatenated_df = concatenated_df.sort_values(by=['time'],ignore_index=True)

In [19]:
concatenated_df

,time,room,co2,light,temp,humidity,pir
0,2013-08-23 15:00:00,511,NaN,NaN,NaN,NaN,0.0
1,2013-08-23 15:01:00,511,NaN,NaN,NaN,NaN,0.0
2,2013-08-23 15:02:00,511,NaN,NaN,NaN,NaN,0.0
3,2013-08-23 15:03:00,511,NaN,NaN,NaN,NaN,0.0
4,2013-08-23 15:04:00,511,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...
602645,2013-09-01 06:58:00,558,NaN,NaN,NaN,NaN,0.0
602646,2013-09-01 06:58:00,726,NaN,NaN,NaN,NaN,0.0
602647,2013-09-01 06:58:00,562,NaN,NaN,NaN,NaN,0.0
602648,2013-09-01 06:58:00,621,NaN,NaN,NaN,NaN,0.0


In [21]:
df_not_NaN = concatenated_df.dropna().reset_index(drop=True)
df_not_NaN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533689 entries, 0 to 533688
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   time      533689 non-null  datetime64[ns]
 1   room      533689 non-null  object        
 2   co2       533689 non-null  float64       
 3   light     533689 non-null  float64       
 4   temp      533689 non-null  float64       
 5   humidity  533689 non-null  float64       
 6   pir       533689 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 28.5+ MB


In [23]:
df_not_NaN.head()

,time,room,co2,light,temp,humidity,pir
0,2013-08-23 23:04:00,511,391.000000,249.000000,22.620000,52.750000,0.0
1,2013-08-23 23:04:00,648,175.000000,191.000000,23.320000,50.320000,0.0
2,2013-08-23 23:04:00,656A,579.000000,176.000000,24.370000,49.900000,30.0
3,2013-08-23 23:05:00,621C,545.125000,267.857143,24.500000,49.030000,0.0
4,2013-08-23 23:05:00,722,370.142857,4.142857,23.111429,48.928571,0.0
